<a href="https://colab.research.google.com/github/atakhadivi/Algorithmic-Trading/blob/master/4H_(10%2C30%2C120%2C30)_full_with_sell_tenkan_kijun.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !python --version

In [ ]:
# !pip3 install -q python-binance
# !wget https://launchpad.net/~mario-mariomedina/+archive/ubuntu/talib/+files/libta-lib0_0.4.0-oneiric1_amd64.deb -qO libta.deb
# !wget https://launchpad.net/~mario-mariomedina/+archive/ubuntu/talib/+files/ta-lib0-dev_0.4.0-oneiric1_amd64.deb -qO ta.deb
# !dpkg -i libta.deb ta.deb
# !pip install ta-lib

In [ ]:
import datetime
from datetime import date
from binance.client import Client
import pandas as pd
# import talib as ta
import numpy as np
import time
import requests
from binance.enums import *
from requests import Session
import sys

In [ ]:
# # api key and api secret
# api_key = 'PCX9QWWPbSTDAe4sifwLRZgaSjyGui0Spnrsdhpx9ABJqn2AP6OE7TAdgKI4vUYO'
# api_secret = 'hWqF3RyVdqI7WoOhX62qwSZnl3vG30XKmMDHqnNNYtOXmfc59DSBuP2Vs7LCLnGO'

In [ ]:
# api key and api secret navid ata_swat@yahoo.com
api_key = 'th5CizfxcOc1AjogJ4AHj6VqGXyKoYayCZl9WMAUc1R0sRL8ASl4IwONI28PCokX'
api_secret = 'hCO7pfeIrFmXTFFcg3hIkmVSQploXuSCS7nAevIIbU9uAH4cwOGaxfw9cpO4wl2K'

In [ ]:
# assest_price = 0.2752
# stop = assest_price + (assest_price * 0.01)
# profit = assest_price - (assest_price * 0.01)
# order_stop = client.futures_create_order(
#     symbol = "XRPUSDT",
#     side = "BUY",
#     type = "STOP_MARKET" ,
#     closePosition = "true",
#     stopPrice = price["XRPUSDT"] % stop
# )
# order_profit = client.futures_create_order(
#     symbol = "XRPUSDT",
#     side = "BUY",
#     type = "TAKE_PROFIT_MARKET" ,
#     closePosition = "true",
#     stopPrice = price["XRPUSDT"] % profit
# )

In [ ]:
symbol_future=[
"BTCUSDT",
"ETHUSDT",
"XRPUSDT",
"ADAUSDT",
"XLMUSDT"
]

In [ ]:
rules={
      "BTCUSDT":"%.3f",
      "ETHUSDT":"%.3f",
      "XRPUSDT":"%.1f",
      "XLMUSDT":"%.0f",
      "ADAUSDT":"%.0f",
}

In [ ]:
price={
      "BTCUSDT":"%.2f",
      "ETHUSDT":"%.2f",
      "XRPUSDT":"%.4f",
      "XLMUSDT":"%.5f",
      "ADAUSDT":"%.5f",
 }

In [ ]:
# telegram bot to send signals
def telegram_bot_sendtext(bot_message):

    bot_token = '1319235780:AAEJsHck45xOkpjAhnJKi4m0c7RQV3VjWHI'
    bot_chatID = '@signalfxcmbinance'
    send_text = 'https://api.telegram.org/bot' + bot_token + '/sendMessage?chat_id=' + bot_chatID + '&parse_mode=Markdown&text=' + bot_message
#https://api.telegram.org/bot1319235780:AAEJsHck45xOkpjAhnJKi4m0c7RQV3VjWHI/sendMessage?chat_id=@signalfxcmbinance&text=salam
    response = requests.get(send_text)

    return response.json()

In [ ]:
def ichimoku(symbol,hist_time,past):
  client = Client(api_key, api_secret)
  d = client.get_historical_klines(symbol,hist_time,past)
  d = pd.DataFrame(d,columns= ['date', 'open', 'high', 'low', 'close', 'volume', 'Close_time','qoute assest volume','number of trades','taker buy base assest volume','taker buy qoute assest volume','ignore'])
  nine_period_high = d['high'].rolling(window= 10).max()
  nine_period_low = d['low'].rolling(window= 10).min()
  d['tenkan_sen'] = (nine_period_high + nine_period_low) /2
  # Kijun-sen (Base Line): (26-period high + 26-period low)/2))
  period26_high = d['high'].rolling(window=30).max()
  period26_low = d['low'].rolling(window=30).min()
  d['kijun_sen'] = (period26_high + period26_low) / 2
  # Senkou Span A (Leading Span A): (Conversion Line + Base Line)/2))
  d['senkou_span_a'] = ((d['tenkan_sen'] + d['kijun_sen']) / 2).shift(30)
  d['senkou_span_a_future'] = ((d['tenkan_sen'] + d['kijun_sen']) / 2)

  # Senkou Span B (Leading Span B): (52-period high + 52-period low)/2))
  period52_high = d['high'].rolling(window=120).max()
  period52_low = d['low'].rolling(window=120).min()
  d['senkou_span_b'] = ((period52_high + period52_low) / 2).shift(30)
  d['senkou_span_b_future'] = ((period52_high + period52_low) / 2)

  # The most current closing price plotted 26 time periods behind (optional)
  d['chikou_span'] = d['close'].shift(-30)
  d = pd.DataFrame(data=d, dtype=np.float64)
  open = d['open'].values
  close = d['close'].values
  high = d['high'].values
  low = d['low'].values
  tenkan_sen = d['tenkan_sen'].values
  kijun_sen = d['kijun_sen'].values
  senkou_span_a = d['senkou_span_a'].values
  senkou_span_a_future = d['senkou_span_a_future'].values
  senkou_span_b = d['senkou_span_b'].values
  senkou_span_b_future = d['senkou_span_b_future'].values
  chikou_span = d['chikou_span'].values
  pric = d['close'].values
  # print(chikou_span[-27])
  return (tenkan_sen,kijun_sen,senkou_span_a,senkou_span_a_future,senkou_span_b,senkou_span_b_future,chikou_span,pric,open,high,low)

In [ ]:
client = Client(api_key, api_secret)

In [ ]:
for i in symbol_future:
  # print(i)
  milli_sec = time.time() * 1000
  sec = int(milli_sec) - 86400000
  tenkan_sen,kijun_sen,senkou_span_a,senkou_span_a_future,senkou_span_b,senkou_span_b_future,chikou_span,pric, open, high, low  = ichimoku(i,hist_time=Client.KLINE_INTERVAL_4HOUR,past="30 day ago UTC")
  if senkou_span_a_future[-1] > senkou_span_b_future[-1]:
    if tenkan_sen[-1] > kijun_sen[-1]:# and tenkan_sen[-2] <= kijun_sen[-2]:
      if pric[-1] > senkou_span_a[-1] and pric[-1] > senkou_span_b[-1]:
        # print(i,('BUY'))
        orders = client.futures_get_open_orders(symbol=i)
        if not orders:
          # text = "{} \n Strong BUY Future Ichimoku 4H ata \n Price: {}" .format(i,pric[-1])
          text = "{} \n Strong BUY Future Ichimoku 4H navid \n Price: {}" .format(i,pric[-1])
          telegram_bot_sendtext(text)
          balance = client.futures_account_balance()
          balance = float(balance[0]['balance'])
          assest_price = pric[-1]
          quantity = 200 / assest_price
          take_profit = assest_price + (assest_price * 0.5)
          stop_price = assest_price - (assest_price * 0.02)
          if (balance > 20):
            # margin = client.futures_change_margin_type(
            #     symbol = i,
            #     marginType = "ISOLATED"
            # )
            leverage = client.futures_change_leverage(
                symbol = i,
                leverage = 1
            )
            order_market = client.futures_create_order(
                symbol = i,
                side = "BUY",
                type = "MARKET",
                quantity = rules[i] % float(quantity)
            )
            order_profit = client.futures_create_order(
                symbol = i,
                side = "SELL",
                type = "TAKE_PROFIT_MARKET" ,
                closePosition = "true",
                stopPrice = price[i] % float(take_profit)
            )
            # print(order_profit)


In [ ]:
for i in symbol_future:
  # print(i)
  milli_sec = time.time() * 1000
  sec = int(milli_sec) - 86400000
  tenkan_sen,kijun_sen,senkou_span_a,senkou_span_a_future,senkou_span_b,senkou_span_b_future,chikou_span,pric, open, high, low  = ichimoku(i,hist_time=Client.KLINE_INTERVAL_4HOUR,past="30 day ago UTC")
  if senkou_span_a_future[-1] < senkou_span_b_future[-1]:
    if tenkan_sen[-1] < kijun_sen[-1]:# and tenkan_sen[-2] <= kijun_sen[-2]:
      if pric[-1] < senkou_span_a[-1] and pric[-1] < senkou_span_b[-1]:
        # print(i,('SELL'))
        orders = client.futures_get_open_orders(symbol=i)
        if not orders:
          # text = "{} \n Strong SELL Future Ichimoku 4H ata \n Price: {}" .format(i,pric[-1])
          text = "{} \n Strong SELL Future Ichimoku 4H navid \n Price: {}" .format(i,pric[-1])
          telegram_bot_sendtext(text)
          balance = client.futures_account_balance()
          balance = float(balance[0]['balance'])
          assest_price = pric[-1]
          quantity = 200 / assest_price
          take_profit = assest_price - (assest_price * 0.5)
          stop_price = assest_price - (assest_price * 0.02)
          if (balance > 20):
            # margin = client.futures_change_margin_type(
            #     symbol = i,
            #     marginType = "ISOLATED"
            # )
            leverage = client.futures_change_leverage(
                symbol = i,
                leverage = 1
            )
            order_market = client.futures_create_order(
                symbol = i,
                side = "SELL",
                type = "MARKET",
                quantity = rules[i] % float(quantity)
            )
            order_profit = client.futures_create_order(
                symbol = i,
                side = "BUY",
                type = "TAKE_PROFIT_MARKET" ,
                closePosition = "true",
                stopPrice = price[i] % float(take_profit)
            )
            # print(order_profit)


In [ ]:
for i in symbol_future:
  # print(i)
  orders = client.futures_get_open_orders(symbol=i)
  if orders and orders[0]['side'] == 'SELL':
    milli_sec = time.time() * 1000
    sec = int(milli_sec) - 86400000
    tenkan_sen,kijun_sen,senkou_span_a,senkou_span_a_future,senkou_span_b,senkou_span_b_future,chikou_span,pric, open, high, low  = ichimoku(i,hist_time=Client.KLINE_INTERVAL_4HOUR,past="30 day ago UTC")
    if tenkan_sen[-1] < kijun_sen[-1] or pric[-1] < senkou_span_a[-1] or pric[-1] < senkou_span_b[-1]:
        # print(i,'cancel long kijun tenkan sen')
        # text = "{} \n kijun tenkan Cancel Long 4H ata \n Price: {}" .format(i,pric[-1])
        text = "{} \n kijun tenkan Cancel Long 4H navid \n Price: {}" .format(i,pric[-1])
        telegram_bot_sendtext(text)
        orders = client.futures_get_open_orders(symbol=i)
        orderId = orders[0]['orderId']
        cancel = client.futures_cancel_order(symbol = i, orderId = orderId)
        assest_price = pric[-1]
        stop = assest_price - (assest_price * 0.01)
        profit = assest_price + (assest_price * 0.01)
        order_stop = client.futures_create_order(
            symbol = i,
            side = "SELL",
            type = "STOP_MARKET" ,
            closePosition = "true",
            stopPrice = price[i] % stop
        )
        # order_profit = client.futures_create_order(
        #     symbol = i,
        #     side  = "SELL",
        #     type = "TAKE_PROFIT_MARKET",
        #     closePosition = "true",
        #     stopPrice = price[i] % profit
        # )
        # print(order_stop)


In [ ]:
for i in symbol_future:
  # print(i)
  orders = client.futures_get_open_orders(symbol=i)
  if orders and orders[0]['side'] == 'BUY':
    milli_sec = time.time() * 1000
    sec = int(milli_sec) - 86400000
    tenkan_sen,kijun_sen,senkou_span_a,senkou_span_a_future,senkou_span_b,senkou_span_b_future,chikou_span,pric, open, high, low  = ichimoku(i,hist_time=Client.KLINE_INTERVAL_4HOUR,past="30 day ago UTC")
    if tenkan_sen[-1] > kijun_sen[-1] or pric[-1] > senkou_span_a[-1] or pric[-1] > senkou_span_b[-1]:
        # print(i,'cancel long kijun tenkan sen')
        # text = "{} \n kijun tenkan Cancel Long 4H ata \n Price: {}" .format(i,pric[-1])
        text = "{} \n kijun tenkan Cancel Long 4H navid \n Price: {}" .format(i,pric[-1])
        telegram_bot_sendtext(text)
        orders = client.futures_get_open_orders(symbol=i)
        orderId = orders[0]['orderId']
        cancel = client.futures_cancel_order(symbol = i, orderId = orderId)
        assest_price = pric[-1]
        stop = assest_price + (assest_price * 0.01)
        profit = assest_price - (assest_price * 0.01)
        order_stop = client.futures_create_order(
            symbol = i,
            side = "BUY",
            type = "STOP_MARKET" ,
            closePosition = "true",
            stopPrice = price[i] % stop
        )
        # order_profit = client.futures_create_order(
        #     symbol = i,
        #     side = "BUY",
        #     type = "TAKE_PROFIT" ,
        #     closePosition = "true",
        #     stopPrice = price[i] % profit
        # )
        # print(order_stop)


In [ ]:
# orders = client.futures_position_information(symbol="ETHUSDT")
# if float(orders[0]['isolatedMargin']) > 1:
#   print(orders)

In [ ]:
# for i in symbol_future:
#   margin = client.futures_change_margin_type(
#       symbol = i,
#       marginType = "ISOLATED"
#   )